## Data cleaning & preprocessing

In [ ]:
# Import necessary libraries
import requests
import pickle
import ast
import pandas as pd
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
import os
from collections import Counter
import numpy as np
import random

### Data Collection

The following function is used to convert the SQL file into a Python dataframe. Each step processes a small batch of the main data to make it computational feasable. Afterwards, the seperate files were combined to the final dataframe.

In [3]:
def process_split_file(split_file_path, batch_size=1000, save_interval=10000):
    """
    Processes a SQL dump file containing INSERT INTO statements, splits the data into batches, 
    and saves each batch to a CSV file.

    Args:
        split_file_path (str): Path to the SQL dump file.
        batch_size (int, optional): Number of rows per batch to process into a DataFrame. Default is 1000.
        save_interval (int, optional): Number of batches to concatenate and save into a single CSV file. Default is 10000.

    Returns:
        None
    """
    # Initialize counters and data storage
    batch_count = 0  # Tracks the number of batches processed since the last save
    file_count = 0   # Tracks the number of saved CSV files
    batch_dfs = []   # List to store DataFrames for concatenation
    batch_data = []  # Temporary list to hold rows of data

    # Open the SQL dump file for reading
    with open(split_file_path, "r", encoding="utf-8") as sql_file:
        # Read the file line by line with a progress bar
        for line in tqdm(sql_file, desc=f"Processing {split_file_path}"):
            # Check if the line starts with an INSERT INTO statement
            if line.strip().startswith("INSERT INTO"):
                # Extract the values part of the INSERT statement
                values = line.split("VALUES")[1].strip().strip(";")
                # Split the values into individual rows
                rows = values.split("),("")
                for row in rows:
                    # Clean up each row and split into individual values
                    cleaned_row = row.strip("()")
                    batch_data.append([value.strip().strip("'") for value in cleaned_row.split(",")])

            # If the batch size is reached, process the data
            if len(batch_data) >= batch_size:
                # Convert the batch data into a DataFrame
                batch_df = pd.DataFrame(batch_data, columns=["NodeID", "pl_from_namespace", "Links"])
                batch_dfs.append(batch_df)
                batch_data = []  # Reset batch data
                batch_count += 1

                # Save data to a CSV file if the save interval is reached
                if batch_count >= save_interval:
                    full_df = pd.concat(batch_dfs, ignore_index=True)
                    save_path = f"{split_file_path}_batch_{file_count}.csv"
                    full_df.to_csv(save_path, index=False)
                    batch_dfs = []  # Reset DataFrame storage
                    batch_count = 0  # Reset batch counter
                    file_count += 1  # Increment file counter

    # Process any remaining data after the loop
    if batch_data:
        batch_df = pd.DataFrame(batch_data, columns=["NodeID", "pl_from_namespace", "Links"])
        batch_dfs.append(batch_df)

    # Save any remaining DataFrames to a CSV file
    if batch_dfs:
        full_df = pd.concat(batch_dfs, ignore_index=True)
        save_path = f"{split_file_path}_batch_{file_count}.csv"
        full_df.to_csv(save_path, index=False)
        print(f"Saved {save_path}")

# Process each split file
for split_file in split_files:
    process_split_file(split_file)

Processing data/splits\split_0.sql: 3402it [02:56, 19.28it/s]


Saved data/splits\split_0.sql_batch_0.csv


Processing data/splits\split_1.sql: 3402it [02:48, 20.23it/s]


Saved data/splits\split_1.sql_batch_0.csv


Processing data/splits\split_2.sql: 3402it [02:46, 20.47it/s]


...

Saved data/splits\split_7.sql_batch_0.csv


Processing data/splits\split_8.sql: 3402it [02:39, 21.35it/s]


Saved data/splits\split_8.sql_batch_0.csv


Processing data/splits\split_9.sql: 3409it [02:42, 20.95it/s]


Saved data/splits\split_9.sql_batch_0.csv


In [ ]:
def combine_processed_files(split_folder, output_file):
    """
    Combines all processed CSV files in a folder into a single CSV file.

    Args:
        split_folder (str): Path to the folder containing processed CSV files.
        output_file (str): Path to save the combined CSV file.

    Returns:
        None
    """
    all_dfs = []  # List to store DataFrames

    # Traverse the folder to find all CSV files
    for root, _, files in os.walk(split_folder):
        for file in files:
            if file.endswith(".csv"):
                df = pd.read_csv(os.path.join(root, file))
                all_dfs.append(df)

    # Concatenate all DataFrames and save to a single CSV
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(output_file, index=False)
    print(f"Final DataFrame saved to {output_file}")

combine_processed_files("data/final_pagelinks.csv")

Final DataFrame saved to data/final_pagelinks.csv


## Data preprocessing

For preprocessing the raw data and enhance computational feasability different steps were performed. First only articles with type = 0 were needed and unnessecary columns removed. Furthermore, self-loops, dead ends and orphaned nodes were dropped.

In [3]:
# Read data
final_df = pd.read_csv("data/final_pagelinks.csv")

# Filter out non-articles
df = final_df[final_df["pl_from_namespace"] == 0]

# Save preprocessing step
df.to_csv("data/reduced_final_pagelinks.csv", index=False)

In [4]:
# Drop the namespace column
tight_df = df.drop(columns="pl_from_namespace")

# Save preprocessing step
tight_df.to_csv("data/reduced_final_pagelinks2.csv", index=False)

In [5]:
# Remove self-loops for complexity reduction
no_self_loops = tight_df[tight_df["NodeID"] != tight_df["Links"]]

# Save preprocessing step
no_self_loops.to_csv("data/no_self_loops_pagelinks.csv", index=False)

In [3]:
# Read data
no_self_loops = pd.read_csv("data/no_self_loops_pagelinks.csv")

# Get set of node and link IDs
node_ids = set(no_self_loops["NodeID"])
link_ids = set(no_self_loops["Links"])

# Filter dead ends
dead_ends = node_ids - link_ids

# Remove dead ends
no_dead_ends = no_self_loops[~no_self_loops["NodeID"].isin(dead_ends)]

# Save preprocessing step
no_dead_ends.to_csv("data/no_dead_ends.csv", index=False)

In [5]:
# Get actual set of node and link IDs
node_ids = set(no_dead_ends["NodeID"])
link_ids = set(no_dead_ends["Links"])

# Filter orphaned nodes
orphaned_nodes = link_ids - node_ids

# Remove orphaned nodes
no_orphaned_nodes = no_dead_ends[~no_dead_ends["Links"].isin(orphaned_nodes)]

# Save preprocessing step
no_orphaned_nodes.to_csv("data/no_orphaned_nodes.csv", index=False)

In [7]:
# Group links by NodeID in a list
grouped_df = no_orphaned_nodes.groupby("NodeID", as_index=False).agg({"Links": list})

# Save preprocessing step
grouped_df.to_csv("data/grouped_df.csv", index=False)

### Building the Graph

The finale dataframe is used to built the Wikipedia Graph.

In [3]:
# Load graph data
graph_df = pd.read_csv("data/grouped_df.csv", index_col=None)
print("Load data done")

# Convert lists in Links to int
graph_df["Links"] = graph_df["Links"].apply(lambda x: [int(item) for item in ast.literal_eval(x)])
print("Convert data done")

Load data done
Convert data done


In [4]:
# Initialize the graph
G = nx.DiGraph() # Directed Graph

# Add nodes and edges from the preprocessed dataset
for _, row in tqdm(graph_df.iterrows(), total=len(graph_df)):
    node = row["NodeID"]
    links = row["Links"]
    G.add_node(node)
    for link in links:
        G.add_edge(link, node)

# Plot descriptive statistics about the graph
print("----- Small Graph -----")
print(f"Number of Nodes smaller Graph: {G.number_of_nodes()}")
print(f"Number of Edges smaller Graph: {G.number_of_edges()}\n")

# Save graph
with open("data/en_wiki_graph.gpickle", "wb") as f:
    pickle.dump(G, f)

100%|██████████| 3770339/3770339 [03:26<00:00, 18239.56it/s]


----- Small Graph -----
Number of Nodes smaller Graph: 4520178
Number of Edges smaller Graph: 101881676



### Validation of the Graph

For validation of the graph strucutre the Giant Component Ratio was calculated.

In [5]:
# Calculate Giant Component Ratio (GCR)
largest_component_size = len(max(nx.strongly_connected_components(G), key=len))
total_nodes = G.number_of_nodes()
GCR = largest_component_size / total_nodes

print(f"Giant Component Ratio (GCR): {GCR:.4f}")

Giant Component Ratio (GCR): 0.4824


### Creation of "Uni-Konstanz"-Graph for ML-tasks

First, the ID of the article "University of Constance" was determine. Second the connected component to this article was identified and used for ML tasks.

In [7]:
def fetch_wikipedia_article_id(title, lang="en"):
    """
    Fetch the article ID of a Wikipedia page given its title using the Wikipedia API.
    
    Parameters:
        title (str): The title of the Wikipedia article.
        lang (str): The language code for the Wikipedia API (default is 'en').
        
    Returns:
        int: The article ID of the Wikipedia page, or None if not found.
    """
    # Construct the URL for the Wikipedia API
    url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "titles": title,
        "format": "json"
    }
    
    try:
        # Make the API request
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Parse the JSON response
        data = response.json()
        pages = data.get("query", {}).get("pages", {})
        
        # Extract the page ID
        for page_id, page_info in pages.items():
            if page_id != "-1":
                return int(page_id)
        return None
    except Exception as e:
        print(f"Error fetching article ID for title '{title}': {e}")
        return None

# Get the article ID for the University of Konstanz
title = "University of Konstanz"
article_id = fetch_wikipedia_article_id(title)
print(f"Article ID for '{title}': {article_id}")

Article ID for 'University of Konstanz': 2562056


In [8]:
# Specify the target article ID to fetch its links
target_node = 2562056

# Find the connected component of the target node (in an undirected version of the graph)
connected_nodes = nx.node_connected_component(G.to_undirected(), target_node)
    
# Create a subgraph from the connected nodes
subgraph = G.subgraph(connected_nodes)
    
# Print information about the original graph and the subgraph
print(f"Original Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"Subgraph: {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges")

# Save the full graph to a file in pickle format
with open("data/kn_graph.gpickle", "wb") as f:
    pickle.dump(G, f)

Original Graph: 4520178 nodes, 101881676 edges
Subgraph: 4515338 nodes, 101879136 edges


# Creation of dataframe for features

Finally, the shortest paths and graph-based metrics, serving as features, were calculated and stored in a dataframe.

In [ ]:
# Load the graph using NetworkX
with open("data/kn_graph.gpickle", "rb") as f:
    subgraph = pickle.load(f)

print("----- Uni-KN Graph -----")
print(f"Number of Nodes smaller Graph: {subgraph.number_of_nodes()}")
print(f"Number of Edges smaller Graph: {subgraph.number_of_edges()}\n")

In [ ]:
# Load or initialize the DataFrame
try:
    # Try loading the DataFrame if it exists
    shortest_path_df = pd.read_csv("data/shortest_path_df.csv")
    print("Loaded existing DataFrame.")
except FileNotFoundError:
    # Initialize an empty DataFrame if not found
    shortest_path_df = pd.DataFrame(columns=["source", "distance"])
    print("Initialized new DataFrame.")

# Assume G is your NetworkX graph
uni_id = 2562056  # Target node ID for Lake Constance

# Compute shortest paths if not already done
if "distance" not in shortest_path_df.columns:
    print("Computing shortest paths...")
    shortest_paths = {}
    for node in tqdm(subgraph.nodes, desc="Computing shortest paths"):
        if node != uni_id:  # Exclude self-loop
            try:
                distance = nx.shortest_path_length(subgraph, source=node, target=uni_id)
                shortest_paths[node] = distance
            except nx.NetworkXNoPath:
                pass  # Skip nodes with no path to Lake Constance

    # Append shortest paths to DataFrame
    data = [{"source": node, "distance": distance} for node, distance in shortest_paths.items()]
    shortest_path_df = pd.DataFrame(data)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("Shortest paths saved.")

# PageRank
if "pagerank_source" not in shortest_path_df.columns:
    print("Computing PageRank...")
    pagerank_scores = nx.pagerank(subgraph)
    shortest_path_df["pagerank_source"] = shortest_path_df["source"].map(pagerank_scores)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("PageRank scores saved.")

# In-Degree
if "in_degree_source" not in shortest_path_df.columns:
    print("Computing In-Degree...")
    in_degrees = dict(subgraph.in_degree())
    shortest_path_df["in_degree_source"] = shortest_path_df["source"].map(in_degrees)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("In-Degrees saved.")

# Out-Degree
if "out_degree_source" not in shortest_path_df.columns:
    print("Computing Out-Degree...")
    out_degrees = dict(subgraph.out_degree())
    shortest_path_df["out_degree_source"] = shortest_path_df["source"].map(out_degrees)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("Out-Degrees saved.")

# HITS
if "hub_score_source" not in shortest_path_df.columns or "authority_score_source" not in shortest_path_df.columns:
    print("Computing HITS...")
    hubs, authorities = nx.hits(subgraph)
    shortest_path_df["hub_score_source"] = shortest_path_df["source"].map(hubs)
    shortest_path_df["authority_score_source"] = shortest_path_df["source"].map(authorities)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("HITS scores saved.")

# Eigenvector Centrality
if "eigenvector_centrality" not in shortest_path_df.columns:
    print("Computing Eigenvector Centrality...")
    eigenvector_centrality = nx.eigenvector_centrality(subgraph, max_iter=100, tol=1e-06)
    shortest_path_df["eigenvector_centrality"] = shortest_path_df["source"].map(eigenvector_centrality)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("Eigenvector Centrality saved.")

# Betweenness Centrality
if "betweenness_centrality" not in shortest_path_df.columns:
    print("Computing Betweenness Centrality...")
    betweenness_centrality = nx.betweenness_centrality(subgraph, k=400, normalized=True)
    shortest_path_df["betweenness_centrality"] = shortest_path_df["source"].map(betweenness_centrality)
    shortest_path_df.to_csv("data/shortest_path_df.csv", index=False)
    print("Betweenness Centrality saved.")

shortest_path_df.head()